In [445]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

In [446]:
# d = range(18202,18962)
match_range = range(18202,18242)

In [447]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [448]:
resington = []
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
#     for s in all_res_data:
#         if 
    all_res_data.append(scripts)
    resington.append(soup)
    #Here we can append only the ones that have 'script'

In [449]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [450]:
# all_res_data = [str(i) for i in all_res_data]

In [451]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [452]:
len(scripts[1])

23

In [453]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [454]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [455]:
# res = requests.get(urls)
# soup = BeautifulSoup(res.content, 'lxml')
# scripts = soup.find_all('script')

In [456]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [457]:
test_list = []
length = len(data_list)
for i in range(length):
    test_list.append(data_list[i]['h'])
# for i in range(length):
#     data_home = data_list[i]['h']

In [458]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
result_list = []
team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    result = []
    team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    xG_list.append(xG)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

NEED TO APPEND it with h_a or whatever so that I can access it

In [467]:
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]
match_id_list = [val for sublist in match_id_list for val in sublist]

#Repeat for all
#Flattening nested lists

In [468]:
col_names = ['match_id','x','y','xG','minute','result','team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,result_list,team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
# df = pd.DataFrame([all_data[0],all_data[1],all_data[2],all_data[3],all_data[4],all_data[5],all_data[6],all_data[7],all_data[8],all_data[9],all_data[10]],index=col_names)
df = df.T
# df = df.astype({'minute':'int'})
# df = df.sort_values(by=['match_id','minute'])

In [460]:
df['away_score'] = 0
df['home_score'] = 0

In [470]:
df

,match_id,x,y,xG,minute,result,team,h_a,player,home_score,away_score
0,18202,"[0.92, 0.850999984741211, 0.9209999847412109, ...","[0.5120000076293946, 0.345, 0.3659999847412109...","[0.3935762047767639, 0.05573191121220589, 0.06...","[41, 45, 45, 46, 51, 65, 71, 80, 80, 80, 84, 3...","[SavedShot, BlockedShot, BlockedShot, BlockedS...","[Crystal Palace, Crystal Palace, Crystal Palac...","[h, h, h, h, h, h, h, h, h, h, h, a, a, a, a, ...","[Odsonne Edouard, Odsonne Edouard, Odsonne Edo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,18202,"[0.8859999847412109, 0.8530000305175781, 0.838...","[0.664000015258789, 0.44299999237060544, 0.439...","[0.06337210536003113, 0.06387635320425034, 0.0...","[0, 23, 23, 30, 31, 55, 56, 57, 71, 21, 28, 38...","[MissedShots, BlockedShot, BlockedShot, Blocke...","[Fulham, Fulham, Fulham, Fulham, Fulham, Fulha...","[h, h, h, h, h, h, h, h, h, a, a, a, a, a, a, ...","[Aleksandar Mitrovic, João Palhinha, João Palh...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,18202,"[0.919000015258789, 0.8369999694824218, 0.8090...","[0.43099998474121093, 0.500999984741211, 0.454...","[0.10186923295259476, 0.08674071729183197, 0.0...","[1, 8, 8, 21, 71, 79, 79, 3, 6, 14, 16, 20, 36...","[Goal, SavedShot, BlockedShot, MissedShots, Bl...","[Bournemouth, Bournemouth, Bournemouth, Bourne...","[h, h, h, h, h, h, h, a, a, a, a, a, a, a, a, ...","[Jefferson Lerma, Dominic Solanke, Dominic Sol...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,18202,"[0.789000015258789, 0.6919999694824219, 0.7390...","[0.3659999847412109, 0.3120000076293945, 0.577...","[0.028160689398646355, 0.01301555149257183, 0....","[12, 18, 19, 23, 25, 26, 28, 69, 71, 79, 79, 8...","[BlockedShot, SavedShot, MissedShots, Goal, Bl...","[Leeds, Leeds, Leeds, Leeds, Leeds, Leeds, Lee...","[h, h, h, h, h, h, h, h, h, h, h, h, a, a, a, ...","[Brenden Aaronson, Patrick Bamford, Tyler Adam...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,18202,"[0.865, 0.9259999847412109, 0.7759999847412109...","[0.39, 0.765, 0.34700000762939454, 0.605999984...","[0.04972551390528679, 0.009584739804267883, 0....","[0, 1, 3, 5, 7, 15, 19, 23, 26, 26, 30, 31, 34...","[BlockedShot, SavedShot, BlockedShot, MissedSh...","[Newcastle United, Newcastle United, Newcastle...","[h, h, h, h, h, h, h, h, h, h, h, h, h, h, h, ...","[Fabian Schär, Dan Burn, Joe Willock, Dan Burn...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
1036,18241,None,None,None,None,None,None,None,None,None,None
1037,18241,None,None,None,None,None,None,None,None,None,None
1038,18241,None,None,None,None,None,None,None,None,None,None
1039,18241,None,None,None,None,None,None,None,None,None,None


In [462]:
df.loc[df['result'] == 'Goal', 'away_score'] = 10